# Exploratory Data Analysis - Telco Churn Dataset

This notebook explores the telco customer churn dataset through three stages:
1. **Raw Data**: Initial data from staging
2. **Feature Engineering**: After dbt transformations
3. **Insights**: Key patterns and relationships

---

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import polars as pl
import seaborn as sns
from plotly.subplots import make_subplots

# Add project root to path
sys.path.append(str(Path.cwd().parent))

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Load environment variables
from dotenv import load_dotenv
load_dotenv(Path.cwd().parent / '.env')

print("✓ Imports loaded")

## 1. Load Data

We'll load both the raw staging data and the engineered features.

In [ ]:
from src.data.loaders import load_features_from_gcs

# Load engineered features from GCS
df_features = load_features_from_gcs(
    "gs://modern-tabular-dev/data/features/churn_features.parquet",
    os.getenv('GCS_KEY_ID'),
    os.getenv('GCS_SECRET'),
)

# Convert to pandas for visualization
df = df_features.to_pandas()

print(f"Dataset shape: {df.shape}")
print(f"Features: {df.shape[1]} columns")
print(f"Samples: {df.shape[0]:,} customers")

## 2. Basic Data Overview

In [ ]:
# Display first few rows
df.head()

In [ ]:
# Data types and missing values
info_df = pd.DataFrame({
    'dtype': df.dtypes,
    'missing': df.isnull().sum(),
    'missing_pct': (df.isnull().sum() / len(df) * 100).round(2),
    'unique': df.nunique()
})

print("\nData Quality Check:")
info_df[info_df['missing'] > 0]  # Show only columns with missing values

In [ ]:
# Summary statistics
df.describe()

## 3. Target Variable Analysis

Understanding the distribution of churn.

In [ ]:
# Churn distribution
churn_counts = df['has_churned'].value_counts()
churn_rate = df['has_churned'].mean()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'bar'}, {'type':'pie'}]])

# Bar chart
fig.add_trace(
    go.Bar(x=['Retained', 'Churned'], y=churn_counts.values, 
           text=churn_counts.values, textposition='auto'),
    row=1, col=1
)

# Pie chart
fig.add_trace(
    go.Pie(labels=['Retained', 'Churned'], values=churn_counts.values),
    row=1, col=2
)

fig.update_layout(title_text=f"Churn Distribution (Rate: {churn_rate:.2%})", showlegend=False)
fig.show()

print(f"\nChurn Rate: {churn_rate:.2%}")
print(f"Churned Customers: {churn_counts[True]:,}")
print(f"Retained Customers: {churn_counts[False]:,}")

## 4. Demographic Analysis

In [ ]:
# Demographics vs Churn
demo_features = ['gender', 'is_senior_citizen', 'has_partner', 'has_dependents']

fig = make_subplots(rows=2, cols=2, subplot_titles=demo_features)

for idx, feature in enumerate(demo_features):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    churn_by_feature = df.groupby(feature)['has_churned'].mean().sort_values(ascending=False)
    
    fig.add_trace(
        go.Bar(x=churn_by_feature.index.astype(str), y=churn_by_feature.values,
               text=[f"{v:.1%}" for v in churn_by_feature.values], textposition='auto'),
        row=row, col=col
    )

fig.update_layout(height=600, title_text="Churn Rate by Demographics", showlegend=False)
fig.update_yaxes(tickformat=".0%")
fig.show()

## 5. Tenure Analysis

How customer tenure relates to churn.

In [ ]:
# Tenure distribution by churn status
fig = px.histogram(df, x='tenure_months', color='has_churned', 
                   nbins=50, barmode='overlay',
                   title='Tenure Distribution by Churn Status',
                   labels={'has_churned': 'Churned', 'tenure_months': 'Tenure (months)'})
fig.update_traces(opacity=0.7)
fig.show()

# Tenure group analysis
tenure_churn = df.groupby('tenure_group')['has_churned'].agg(['mean', 'count'])
print("\nChurn by Tenure Group:")
print(tenure_churn)

## 6. Financial Features Analysis

In [ ]:
# Monthly charges distribution
fig = px.box(df, x='has_churned', y='monthly_charges', 
             title='Monthly Charges Distribution by Churn Status',
             labels={'has_churned': 'Churned', 'monthly_charges': 'Monthly Charges ($)'})
fig.show()

# Key stats
print("\nMonthly Charges by Churn Status:")
print(df.groupby('has_churned')['monthly_charges'].describe())

In [ ]:
# Charge velocity (rate of price change)
fig = px.histogram(df, x='charge_velocity', color='has_churned',
                   nbins=50, barmode='overlay',
                   title='Charge Velocity Distribution (Price Change Rate)',
                   labels={'charge_velocity': 'Charge Velocity ($)', 'has_churned': 'Churned'})
fig.update_traces(opacity=0.7)
fig.show()

## 7. Contract & Billing Analysis

In [ ]:
# Contract type impact
contract_analysis = df.groupby('contract_type').agg({
    'has_churned': ['mean', 'count'],
    'monthly_charges': 'mean',
    'tenure_months': 'mean'
}).round(2)

contract_analysis.columns = ['churn_rate', 'count', 'avg_monthly_charge', 'avg_tenure']
contract_analysis['churn_rate'] = contract_analysis['churn_rate'].apply(lambda x: f"{x:.1%}")

print("Contract Type Analysis:")
print(contract_analysis)

# Visualization
fig = px.bar(df.groupby('contract_type')['has_churned'].mean().sort_values(ascending=False),
             title='Churn Rate by Contract Type',
             labels={'value': 'Churn Rate', 'contract_type': 'Contract Type'})
fig.update_yaxes(tickformat=".0%")
fig.show()

## 8. Service Usage Analysis

In [ ]:
# Internet service type
internet_churn = df.groupby('internet_service')['has_churned'].mean().sort_values(ascending=False)

fig = px.bar(internet_churn, 
             title='Churn Rate by Internet Service Type',
             labels={'value': 'Churn Rate', 'internet_service': 'Internet Service'})
fig.update_yaxes(tickformat=".0%")
fig.show()

print("\nInternet Service Analysis:")
print(df.groupby('internet_service')['has_churned'].agg(['mean', 'count']))

In [ ]:
# Total services subscribed
fig = px.box(df, x='total_services_count', y='monthly_charges', color='has_churned',
             title='Monthly Charges by Number of Services and Churn Status',
             labels={'total_services_count': 'Number of Services', 
                     'monthly_charges': 'Monthly Charges ($)',
                     'has_churned': 'Churned'})
fig.show()

# Churn by service count
services_churn = df.groupby('total_services_count')['has_churned'].mean()
print("\nChurn Rate by Number of Services:")
print(services_churn)

## 9. Engineered Features Analysis

Examining our custom features from dbt transformations.

In [ ]:
# Churn risk score distribution
fig = px.histogram(df, x='churn_risk_score', color='has_churned',
                   nbins=20, barmode='overlay',
                   title='Churn Risk Score Distribution',
                   labels={'churn_risk_score': 'Churn Risk Score', 'has_churned': 'Churned'})
fig.update_traces(opacity=0.7)
fig.show()

# Risk score effectiveness
risk_analysis = df.groupby('churn_risk_score')['has_churned'].mean()
print("\nActual Churn Rate by Risk Score:")
print(risk_analysis)

In [ ]:
# Engagement score
fig = px.box(df, x='has_churned', y='engagement_score',
             title='Engagement Score by Churn Status',
             labels={'engagement_score': 'Engagement Score', 'has_churned': 'Churned'})
fig.show()

print("\nEngagement Score Statistics:")
print(df.groupby('has_churned')['engagement_score'].describe())

## 10. Correlation Analysis

In [ ]:
# Select numeric features for correlation
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_features = [col for col in numeric_features if col != 'customer_id']

# Correlation with target
correlations = df[numeric_features].corrwith(df['has_churned']).sort_values(ascending=False)

fig = px.bar(correlations.drop('has_churned'), 
             title='Feature Correlation with Churn',
             labels={'value': 'Correlation', 'index': 'Feature'})
fig.update_layout(showlegend=False)
fig.show()

print("\nTop 10 Correlations with Churn:")
print(correlations.head(10))

In [ ]:
# Correlation heatmap (top features)
top_features = correlations.head(15).index.tolist()
corr_matrix = df[top_features].corr()

fig = px.imshow(corr_matrix, 
                text_auto='.2f',
                aspect='auto',
                title='Correlation Matrix - Top Features',
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1)
fig.show()

## 11. Key Insights Summary

### Main Findings:

1. **Churn Rate**: Overall churn rate of ~26.5%

2. **Contract Type**: Month-to-month contracts show highest churn risk

3. **Tenure**: New customers (< 12 months) have significantly higher churn

4. **Services**: Fiber optic customers churn more despite higher revenue

5. **Engagement**: Lower service adoption correlates with higher churn

6. **Engineered Features**: Risk score shows strong predictive signal

### Recommendations:
- Focus retention efforts on new customers
- Incentivize longer-term contracts
- Investigate fiber optic service quality issues
- Promote multi-service bundles


In [ ]:
# Final summary statistics
summary = pd.DataFrame({
    'Metric': ['Total Customers', 'Churned Customers', 'Churn Rate', 
               'Avg Tenure (months)', 'Avg Monthly Charges'],
    'Value': [
        f"{len(df):,}",
        f"{df['has_churned'].sum():,}",
        f"{df['has_churned'].mean():.2%}",
        f"{df['tenure_months'].mean():.1f}",
        f"${df['monthly_charges'].mean():.2f}"
    ]
})

print("\n" + "="*50)
print("DATASET SUMMARY")
print("="*50)
print(summary.to_string(index=False))
print("="*50)